In [1]:
import random
from random import *

In [14]:
"""
#  #  #  #  #  #  1  #  #  #  #  #  #
#  #  #  #  #  2  3  4  #  #  #  #  #
#  #  #  #  5  6  7  8  9  #  #  #  #
#  #  #  10 11 12 13 14 15 16 #  #  #
#  #  17 18 19 20 21 22 23 24 25 #  #
#  26 27 28 29 30 31 32 33 34 35 36 #
37 38 39 40 41 42 -- 44 45 46 47 48 49
#  50 51 52 53 54 55 56 57 58 59 60 #
#  #  61 62 63 64 65 66 67 68 69 #  #
#  #  #  70 71 72 73 74 75 76 #  #  #
#  #  #  #  77 78 79 80 81 #  #  #  #
#  #  #  #  #  82 83 84 #  #  #  #  #
#  #  #  #  #  #  85 #  #  #  #  #  #
"""

END_TURN = -1
ACTIONS = []

for i in range(7):
    if i != 0:
        ACTIONS.append((0, i))
        ACTIONS.append((0, -i))

    for j in range(6 - i):
        ACTIONS.append((1 + j, i))
        ACTIONS.append((-1 - j, i))
        
        if i != 0:
            ACTIONS.append((1 + j, -i))
            ACTIONS.append((-1 - j, -i))

ACTIONS.append(END_TURN)
print(len(ACTIONS))

85


In [15]:
REWARD_MOB = 16
REWARD_END_TURN = -4
REWARD_MOVE = -1
REWARD_IMPOSSIBLE = -128
REWARD_END_FIGHT = 64

EMPTY_CELL = 0
WALL = 1
USABLE_CELL = 2
PLAYER = 3
MONSTER = 4
INVO = 5
BOSS = 6
UNDEFINED = -1

In [7]:
l = 20
h = 14

dict_swap_dofus_to_pos = {}
dict_swap_pos_to_dofus  = {}

for j in range(h):
    
    x_start = 13 - j
    
    for i in range(l):
        
        dict_swap_dofus_to_pos[(j, i, False)] = (x_start + i, i + j)
        dict_swap_dofus_to_pos[(j, i, True)] = (x_start + i, i + j + 1)
        
        dict_swap_pos_to_dofus[(x_start + i, i + j)] = (j, i, False)
        dict_swap_pos_to_dofus[(x_start + i, i + j + 1)] = (j, i, True)

In [12]:
def get_map(path):
    
    dict_map = {}
    array_map = []
    
    with open(path, 'r') as f:
        lines = f.readlines()
        
        for line in lines:
            array_map.append(list(map(int, line.split(','))))
    
    for pos in dict_swap_pos_to_dofus.keys():
        dict_map[pos] = array_map[pos[0]][pos[1]]
    
    return dict_map

In [13]:
list_map = []

for i in range(5):
    list_map.append(get_map(f"./data/map_array/map_{i + 1}.txt"))

In [ ]:
def get_random_start_map(dict_map):
    
    list_keys_player = []
    list_keys_mob = []
    
    for key in dict_map.keys():
        if dict_map[key] == 3:
            list_keys_player.append(key)
        elif dict_map[key] == 4:
            list_keys_mob.append(key)
            
    i = random.randint(0, len(list_keys_player) - 1)
    list_keys_player.remove(list_keys_player[i])
    
    for _ in range(4):
        i = random.randint(0, len(list_keys_mob) - 1)
        list_keys_mob.remove(list_keys_mob[i])
        
    for pos in list_keys_player:
        dict_map[pos] = 2
        
    for pos in list_keys_mob:
        dict_map[pos] = 2

In [ ]:
def get_player_pos(dict_map):
    
    for pos in dict_map.keys():
        if dict_map[pos] == 3:
            return pos
        
    return -1

In [ ]:
def mob_turn(dict_map, boss_map=False):
    return dict_map

In [ ]:
def map_contains_mob(dict_map):
    
    for pos in dict_map.keys():
        if dict_map[pos] == MONSTER or dict_map[pos] == INVO or dict_map[pos] == BOSS:
            return True
    
    return False

In [ ]:
def count_pm_to_pos(dict_map, player_pos, expected_pos, max_pm):
    return 3

In [ ]:
class Environment:
    
    def __init__(self):
        
        self.map_index = random.randint(0, 4)
        self.map = get_random_start_map(list_map[self.map_index])
        
        self.max_pa = 6
        self.max_pm = 3
        
        self.pa = self.max_pa
        self.pm = self.max_pm
        
    def do(self, state, action):
        
        clicked = ACTIONS[action]
        player_pos = get_player_pos(state)
        
        if player_pos == -1:
            exit()
        
        if clicked == END_TURN:
            self.pa = self.max_pa
            self.pm = self.max_pm
            return mob_turn(self.map, boss_map=self.map_index == 4), REWARD_END_TURN
        
        clicked_pos = player_pos[0] + clicked[0], player_pos[1] + clicked[1]
        
        if state[clicked_pos] == EMPTY_CELL or state[clicked_pos] == WALL or state[clicked_pos] == UNDEFINED:
            return state, REWARD_IMPOSSIBLE
        elif state[clicked_pos] == MONSTER or state[clicked_pos] == INVO or state[clicked_pos] == BOSS:
            state[clicked_pos] = 2
            if not map_contains_mob(state):
                return state, REWARD_IMPOSSIBLE
            return state, REWARD_MOB
        elif state[clicked_pos] == USABLE_CELL:
            used = count_pm_to_pos(state, player_pos, clicked_pos, self.pm)
            
            if used == -1:
                return state, REWARD_IMPOSSIBLE
            
            self.pm -= used
            return state, REWARD_MOVE
